In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import itertools
from matplotlib.colors import LogNorm
from scipy.stats import binned_statistic
from scipy.optimize import curve_fit

In [ ]:
lambda_d0, dc_d0, c_d0 = np.genfromtxt("data/measurement/DiBellaFiedlerD0.txt", unpack=True)
lambda_d60, dc_d60, c_d60 = np.genfromtxt("data/measurement/DiBellaFiedlerD60.txt", unpack=True)
lambda_l0, dc_l0, c_l0 = np.genfromtxt("data/measurement/DiBellaFiedlerL0.txt", unpack=True)
lambda_l60, dc_l60, c_l60 = np.genfromtxt("data/measurement/DiBellaFiedlerL60.txt", unpack=True)

In [ ]:
plt.plot(lambda_d0,abs(c_d0-dc_d0))
#plt.plot(lambda_d60,abs(c_d60-dc_d60))
plt.plot(lambda_l0,abs(c_l0-dc_l0))
#plt.plot(lambda_l60,abs(c_l60-dc_l60))


In [ ]:
h_deg = np.arange(-20,101,11)
v_deg = np.arange(-6,90,12)
single = np.array([["C1_dc_h90_v90"],["C1_att_h90_v90"]])

df_dc = pd.read_csv(f"data/measurement/DiBellaFiedlerRadial/DarkCounts_h=2deg_v=6deg.txt", sep="\t", header=0, usecols=[0],  names=["lambda"])
df_att = pd.read_csv(f"data/measurement/DiBellaFiedlerRadial/Attenuation_h=2deg_v=6deg_x=0mm.txt", sep="\t", header=0, usecols=[0] ,  names=["lambda"])

for i,j in itertools.product(h_deg,v_deg):
    cols_dc = [f"C1_dc_h{i}_v{j}"]
    cols_att = [f"C1_att_h{i}_v{j}"]
    
    df_dc[cols_dc] = pd.read_csv(f"data/measurement/DiBellaFiedlerRadial/DarkCounts_h={i}deg_v={j}deg.txt", sep="\t", header=0, usecols=[1],  names=cols_dc)
    df_att[cols_att] = pd.read_csv(f"data/measurement/DiBellaFiedlerRadial/Attenuation_h={i}deg_v={j}deg_x=0mm.txt", sep="\t", header=0, usecols=[1] ,  names=cols_att)

df_dc[single[0]] = pd.read_csv(f"data/measurement/DiBellaFiedlerRadial/DarkCounts_h=90deg_v=90deg.txt", sep="\t", header=0, names=single[0], usecols=[1])
#df_att[single[1]] = pd.read_csv(f"data/measurement/DiBellaFiedlerRadial/Attenuation_h=90deg_v=90deg_x=0mm.txt", sep="\t", header=0 ,names=single[1], usecols=[1])

df_att.loc[750]


In [ ]:
x= abs(df_att.sum(axis=0).to_numpy() - df_dc.sum(axis=0).to_numpy()[1:])

a = abs(df_att.loc[750].to_numpy() - df_dc.loc[750].to_numpy()[:-1])
a = a[1:].reshape((len(v_deg),len(h_deg)))
x = x[1:].reshape((len(v_deg),len(h_deg)))

plt.matshow(a,extent=(-20,90,78,-6))
plt.colorbar()

In [ ]:
linplt = np.linspace(-90,90,200)
gauss = lambda x,a,mu,sigma,c: a/np.sqrt(2*np.pi*sigma**2) * np.exp(-(x-mu)**2/sigma**2) + c

gauss_par, gauss_cov = curve_fit(gauss,v_deg,a.sum(axis=1),p0=[500,0,20,20],sigma=None,absolute_sigma=False)

plt.plot(h_deg,a.sum(axis=0))#,"x")
#plt.plot(linplt,gauss(linplt,*gauss_par))

gauss_par

#plt.ylim((0,180))

In [ ]:
lambda_s = df_att["lambda"].to_numpy()
b = df_att["C1_att_h2_v30"].to_numpy() - df_dc["C1_dc_h2_v30"].to_numpy()
plt.plot(lambda_s,b)